# How to load data, and loading times

In [20]:
import pandas as pd
import time

## Load listings
For the listings, we can open the xlsx in Excel and convert the "listings" worksheet to a csv to it can then be loaded into Pandas no problem.

In [21]:

print("Loading listings...")

start = time.time()

print("Texas...")
texas_listings = pd.read_csv("../data/raw/texas_listings.csv", encoding="unicode_escape", low_memory=False)
print("Elapsed time: ", time.time() - start)

print("Florida...")
florida_listings = pd.read_csv("../data/raw/florida_listings.csv", encoding="unicode_escape", low_memory=False)
print("Elapsed time: ", time.time() - start)

print("California...")
california_listings = pd.read_csv("../data/raw/california_listings.csv", encoding="unicode_escape", low_memory=False)
print("Elapsed time: ", time.time() - start)

print("\nShapes of data:")
print("Texas: ", texas_listings.shape)
print("Florida: ", florida_listings.shape)
print("California: ", california_listings.shape)

Loading listings...
Texas...
Elapsed time:  0.7278032302856445
Florida...
Elapsed time:  1.3965225219726562
California...
Elapsed time:  2.0300753116607666

Shapes of data:
Texas:  (11882, 106)
Florida:  (10229, 106)
California:  (8111, 106)


## Load and Process Reviews

The reviews have newlines, tabs, and returns in the text fields, which corrupts the CSV format if you export directly from excel. Therefore, we need to clean the data before we can load it. We'll then export the results to a csv so we don't need to do this every time.

In [27]:
import pandas as pd

def process_reviews(fp, target_fp, target_worksheet="Reviews"):
    """Process reviews from excel file and write to csv file for later, faster loading.

    Args:
        fp (str): The fp to the xlsx.
        target_fp (str): Target csv file path.
        target_worksheet (str, optional): The worksheet name to read from. Defaults to "Reviews".
    """

    print("Reading", fp)
    reviews = pd.read_excel(fp, sheet_name=target_worksheet)

    print("Removing line endings")
    reviews.comments = reviews.comments.str.replace("\r", " ").replace("\n", " ").replace("\t", " ")

    print("Writing to", target_fp)
    with open(target_fp, "w", encoding="utf-8") as f:
        f.write(reviews.to_csv(index=False))

In [15]:
process_reviews("../data/raw/TX Austin - All Data.xlsx", "../data/raw/texas_reviews.csv")

Reading ../data/raw/TX Austin - All Data.xlsx
Removing line endings
Writing to ../data/raw/texas_reviews.csv


In [23]:
process_reviews("../data/raw/CA San Francisco - All Data.xlsx", "../data/raw/california_reviews.csv")

Reading ../data/raw/CA San Francisco - All Data.xlsx
Removing line endings
Writing to ../data/raw/california_reviews.csv


In [26]:
process_reviews("../data/raw/FL Fort Lauderdale - All Data.xlsx", "../data/raw/florida_reviews.csv", target_worksheet="reviews")

Reading ../data/raw/FL Fort Lauderdale - All Data.xlsx
Removing line endings
Writing to ../data/raw/florida_reviews.csv


Let's check that the new CSVs are valid by loading them into Pandas and check the runtimes.

In [28]:
now = time.time()

print("Loading reviews...")
print("Texas...")
texas_reviews = pd.read_csv("../data/raw/texas_reviews.csv")
print("Elapsed time: ", time.time() - now)

print("Florida...")
florida_reviews = pd.read_csv("../data/raw/florida_reviews.csv")
print("Elapsed time: ", time.time() - now)

print("California...")
california_reviews = pd.read_csv("../data/raw/california_reviews.csv")
print("Elapsed time: ", time.time() - now)

print("\nShapes of data:")
print("Texas: ", texas_reviews.shape)
print("Florida: ", florida_reviews.shape)
print("California: ", california_reviews.shape)




Loading reviews...
Texas...
Elapsed time:  1.0305256843566895
Florida...
Elapsed time:  1.6577794551849365
California...
Elapsed time:  3.3999249935150146

Shapes of data:
Texas:  (332098, 6)
Florida:  (195857, 6)
California:  (366643, 6)
